In [ ]:
#import shutil
#shutil.rmtree('/content/train/')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -qq "/content/drive/MyDrive/pokemon/train.zip" -d "/content/train"
!unzip -qq "/content/drive/MyDrive/pokemon/test.zip" -d "/content/test"

# 새 섹션

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, Flatten, MaxPool2D, Dense, ReLU, BatchNormalization, Add, Dropout, GlobalAveragePooling2D
from tensorflow.keras.datasets import mnist
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow import pad
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from PIL import Image
import os

# 포켓몬 이미지가 저장된 기본 디렉토리
base_dir = '/content'

# Image data generator with some data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

# Flow training images in batches
train_generator = train_datagen.flow_from_directory(base_dir + "/train", target_size=(150, 150), batch_size=32, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(base_dir + "/test", target_size=(150, 150), batch_size=32, class_mode='categorical')

base_model = MobileNet(include_top=False, weights='imagenet', input_shape=(150, 150, 3))

model = tf.keras.models.Sequential([
    base_model,
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 훈련
history = model.fit(train_generator, epochs=5)



Found 1329 images belonging to 20 classes.
Found 2200 images belonging to 151 classes.


17225924/17225924 [==============================] - 1s 0us/step
Epoch 1/5
20/42 [=============>................] - ETA: 9s - loss: 8.5920 - accuracy: 0.2976 

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


42/42 [==============================] - 46s 498ms/step - loss: 4.8519 - accuracy: 0.4447
Epoch 2/5
42/42 [==============================] - 21s 512ms/step - loss: 0.9427 - accuracy: 0.7299
Epoch 3/5
42/42 [==============================] - 23s 554ms/step - loss: 0.6161 - accuracy: 0.8269
Epoch 4/5
42/42 [==============================] - 22s 534ms/step - loss: 0.5038 - accuracy: 0.8457
Epoch 5/5
42/42 [==============================] - 23s 544ms/step - loss: 0.4322 - accuracy: 0.8751


In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functi  (None, 4, 4, 1024)        3228864   
 onal)                                                           
                                                                 
 flatten_2 (Flatten)         (None, 16384)             0         
                                                                 
 dropout_2 (Dropout)         (None, 16384)             0         
                                                                 
 dense_5 (Dense)             (None, 512)               8389120   
                                                                 
 dense_6 (Dense)             (None, 20)                10260     
                                                                 
Total params: 11628244 (44.36 MB)
Trainable params: 11606356 (44.27 MB)
Non-trainable params: 21888 (85.50 KB)
_________

In [ ]:
print(train_generator.class_indices)

{'1': 0, '10': 1, '16': 2, '19': 3, '21': 4, '23': 5, '25': 6, '27': 7, '35': 8, '37': 9, '39': 10, '4': 11, '41': 12, '43': 13, '50': 14, '52': 15, '54': 16, '58': 17, '66': 18, '7': 19}


In [ ]:
import sklearn
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns


# 모델 테스트
test_loss, test_accuracy = model.evaluate(test_generator) # test_generator
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
'''
# Visualize loss and accuracy
plt.figure(figsize=(12, 4))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()
'''

7/7 [==============================] - 8s 1s/step - loss: 4.8279 - accuracy: 0.2740
Test Accuracy: 27.40%


"\n# Visualize loss and accuracy\nplt.figure(figsize=(12, 4))\n\n# Loss plot\nplt.subplot(1, 2, 1)\nplt.plot(history.history['loss'], label='Training Loss')\nplt.title('Training Loss')\nplt.xlabel('Epochs')\nplt.ylabel('Loss')\nplt.legend()\n\nplt.show()\n"

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from google.colab import files

# 입력 이미지 업로드
uploaded_test_images = files.upload()

# 업로드한 이미지 파일명 가져오기
test_image_name = list(uploaded_test_images.keys())[0]  # 첫 번째 업로드한 이미지 이름

# 이미지 로드 및 사이즈 조정
test_image = load_img(test_image_name, target_size=(150, 150))
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

# 클래스 예측
result = model.predict(test_image)
print(result)
predicted_class = np.argmax(result, axis=1)
print(predicted_class)

# 해당 클래스의 레이블 가져오기
label_map = train_generator.class_indices
predicted_label = list(label_map.keys())[list(label_map.values()).index(predicted_class)]

In [ ]:
print(train_generator.classes)

In [ ]:
import pandas as pd
# 구글 드라이브 csv 파일 경로 파싱
url = "https://drive.google.com/file/d/10ohxV2CbKEgCYgDS1tGlyqsYbMGaxmSK/view?usp=sharing"
path='https://drive.google.com/uc?id='+url.split('/')[-2]

# 포켓몬 CSV 파일 읽고 정보 추출
try:
    pokemon_df = pd.read_csv(path)
except UnicodeDecodeError:
    pokemon_df = pd.read_csv(path, encoding='ISO-8859-1')  # 엑셀 파일 깨짐 해결 encoding='ISO-8859-1'

# 예측된 클래스 번호를 기반으로 포켓몬 정보 찾기
predicted_label = predicted_class[0]
generator_class = train_generator.class_indices
predicted_no = int(str([key for key, value in generator_class.items() if value == predicted_label][0]))
predicted_pokemon_info = pokemon_df[pokemon_df['No.'] == predicted_no].iloc[0]

# 예측 확률 계산
predicted_probability = np.max(result) * 100

# Type1 및 Type2 정보 가져오기 (없는 경우 'None'으로 처리)
type1 = predicted_pokemon_info['type_1'] if not pd.isna(predicted_pokemon_info['type_1']) else 'None'
type2 = predicted_pokemon_info['type_2'] if not pd.isna(predicted_pokemon_info['type_2']) else 'None'

# 결과 출력
print(f"{predicted_probability:.2f}% 확률로 {predicted_no}번 포켓몬 \"{predicted_pokemon_info['name']}\"이고, Type1은 \"{type1}\"이고, Type2는 \"{type2}\"입니다.")

73.76% 확률로 4번 포켓몬 "Charmander"이고, Type1은 "Fire"이고, Type2는 "None"입니다.
